In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
importlib.reload(kgs)
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


In [ ]:
@dataclass
class MegaDynamics(kgs.BaseClass):
    # Configuration
    n_trees = 40
    n_ensembles = 1000
    n_rounds = 5
    seed = 42
    plot_interval = None

    # Hyperparameters
    size_setup = 0.5 # will be mulitiplied by sqrt(n_trees)
    duration_init = 10.
    duration_compact = 200.
    duration_final = 10. # careful - lowering this can seem to make things better, but actually can mean overlapping solutions
    dt = 0.02
    scaling_area_start = 0.3
    scaling_area_end = 0.001
    scaling_boundary = 5.
    scaling_overlap = 1. # recommend to keep this fixed
    use_boundary_distance = True

    # Results
    sizes = None # n_ensemble rows, n_round columns
    best_size = np.inf
    best_xyt = None

    #@kgs.profile_each_line
    def run_simulation(self):
        # Initial configuration
        size_setup_scaled = self.size_setup * np.sqrt(self.n_trees)
        xyt = np.random.default_rng(seed=self.seed).uniform(-0.5, 0.5, size=(self.n_ensembles, self.n_trees, 3))
        xyt = xyt * [[[size_setup_scaled, size_setup_scaled, np.pi]]]
        xyt = np.array(xyt, dtype=np.float32)
        h = np.array([[2*size_setup_scaled,0,0]]*self.n_ensembles, dtype=np.float32)
        cost = pack_cost.CostCompound(costs=[pack_cost.AreaCost(scaling=0.),
                                             pack_cost.BoundaryDistanceCost(scaling=self.scaling_boundary), 
                                             pack_cost.CollisionCostOverlappingArea(self.scaling_overlap)])
        #cost = pack_cost.CostCompound(costs=[pack_cost.AreaCost(scaling=0.)])
        if not self.use_boundary_distance:
            cost.costs[1] = pack_cost.BoundaryCost(scaling=self.scaling_boundary)


        plt.ioff()
        if self.plot_interval is not None:
            plt.ion()
            fig, ax = plt.subplots(figsize=(8, 8))
            tree_list = kgs.TreeList()
            #print(xyt[0].shape)
            #tree_list.xyt = xyt[0]
            #pack_vis.visualize_tree_list(tree_list, ax=ax, h=size_setup_scaled)
            #display(fig)
            #clear_output(wait=True)

        sol = kgs.SolutionCollection()
        xyt = cp.array(xyt)
        h = cp.array(h)
        
        # Preallocate gradient arrays
        total_cost = cp.zeros(self.n_ensembles, dtype=cp.float32)
        total_grad = cp.zeros_like(xyt)
        bound_grad = cp.zeros_like(h)

        t_total = np.float32(0.)
        dt = np.float32(self.dt)
        phase = 'init'
        t_this_phase = np.float32(0.)
        t_last_plot = np.float32(0.)
        rounds_done = 0
        self.sizes = np.zeros((self.n_ensembles, self.n_rounds), dtype=np.float32)
        while True:
            if phase == 'compact':
                frac = t_this_phase / self.duration_compact
                start = self.scaling_area_start
                end = self.scaling_area_end
                area_scaling = start * (end / start) ** frac
                cost.costs[0].scaling = area_scaling
            else:
                cost.costs[0].scaling = 0.
            sol.xyt = xyt
            sol.h = h
            cost.compute_cost(sol, total_cost, total_grad, bound_grad)
            xyt -= dt * total_grad
            h -= dt * bound_grad
            t_total += dt
            t_this_phase += dt
            if self.plot_interval is not None and t_total - t_last_plot >= self.plot_interval*0.999:
                t_last_plot = t_total                
                ax.clear()
                ax.set_aspect('equal', adjustable='box')
                tree_list.xyt = cp.asnumpy(xyt[0])
                pack_vis.visualize_tree_list(tree_list, ax=ax, h=cp.asnumpy(h[0,0]))
                ax.set_title(f'Time: {t_total:.2f}, Round:{rounds_done+1}, Phase: {phase}, Area scaling: {cost.costs[0].scaling:.4f}, Cost: {total_cost[0]:.4f}')
                display(fig)
                clear_output(wait=True)       

            if phase == 'init' and t_this_phase >= self.duration_init:
                phase = 'compact'
                t_this_phase = 0.                
            elif phase == 'compact' and t_this_phase >= self.duration_compact:
                phase = 'final'
                t_this_phase = 0.
            elif phase == 'final' and t_this_phase >= self.duration_final:
                

                rounds_done += 1
                if rounds_done >= self.n_rounds:
                    break
                
                phase = 'compact'
                t_this_phase = 0.                  

        # If a temporary plotting figure was created, close it to release resources
        #if self.plot_interval is not None:
        #    plt.close(fig)


In [ ]:
%%time
kgs.profiling=False
kgs.debugging_mode = 1
runner_list = []
base_runner = MegaDynamics()
base_runner.n_trees = 40
base_runner.n_ensembles = 10000
base_runner.n_rounds = 1
base_runner.duration_compact = 50.
base_runner.plot_interval = None
base_runner.run_simulation()

CPU times: user 17.9 s, sys: 597 ms, total: 18.5 s
Wall time: 19.5 s
